# L5: LLM과 채팅하기! 💬
LLM은 Large Language Model의 약자입니다.

당신의 HuggingFace API 키, 그리고 관련된 파이썬 라이브러리를 불러오세요

In [ ]:
import os
import io
import IPython.display
from PIL import Image
import base64 
import requests 
requests.adapters.DEFAULT_TIMEOUT = 60

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # 로컬의 .env 파일을 읽어 옵니다
hf_api_key = os.environ['HF_API_KEY']

In [ ]:
# Helper 함수
import requests, json
from text_generation import Client

# text_generation 라이브러리에서 FalcomLM-instruct 엔드포인트를 불러옵니다
client = Client(os.environ['HF_API_FALCOM_BASE'], headers={"Authorization": f"Basic {hf_api_key}"}, timeout=120)

## LLM과 대화하는 앱 만들기!

[🤗 Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard) 에서 best 랭킹을 차지하는 오픈 소스 LLM,  
 `falcon-40b-instruct`의 엔드포인트 [Inference Endpoint](https://huggingface.co/inference-endpoints)를 사용합니다.

이를 로컬에서 실행하기 위해서는 [Transformers library](https://huggingface.co/docs/transformers/index) 또는 [text-generation-inference](https://github.com/huggingface/text-generation-inference) 를 사용해야 합니다.

In [ ]:
prompt = "Has math been invented or discovered?"
client.generate(prompt, max_new_tokens=256).generated_text

In [ ]:
# Lesson 2에서 배운 내용을 활용해 봅시다!
import gradio as gr
def generate(input, slider):
    output = client.generate(input, max_new_tokens=slider).generated_text
    return output

demo = gr.Interface(fn=generate, inputs=[gr.Textbox(label="Prompt"), gr.Slider(label="Max new tokens", value=20,  maximum=1024, minimum=1)], outputs=[gr.Textbox(label="Completion")])
gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT1']))

## `gr.Chatbot()`

In [ ]:
import random

def respond(message, chat_history):
        # 아직 LLM을 사용하는 것은 아닙니다.
        # 단지 사전에 정의된 메세지로 랜덤하게 답변하게끔 합니다.
        bot_message = random.choice(["Tell me more about it", 
                                     "Cool, but I'm not interested", 
                                     "Hmmmm, ok then"]) 
        chat_history.append((message, bot_message))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) # 노트북 사이즈에 맞추기 위한 수치일 뿐입니다
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console") # 클리어 버튼

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) # 제출하려면 엔터를 누르세요
gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT2']))

In [ ]:
# 우리가 원하는 것은 채팅 형식이므로 multi-turn 대화가 가능하도록 수정합니다.
def format_chat_prompt(message, chat_history): 
    prompt = ""
    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

def respond(message, chat_history):
        formatted_prompt = format_chat_prompt(message, chat_history)
        bot_message = client.generate(formatted_prompt,
                                     max_new_tokens=1024,
                                     # 문장 끝을 나타내는 <|endoftext|> 토큰으로 구분
                                     stop_sequences=["\nUser:", "<|endoftext|>"]).generated_text
        chat_history.append((message, bot_message))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) # 노트북 사이즈에 맞추기 위한 수치일 뿐입니다
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console") # 클리어 버튼

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) # 제출을 위해서 엔터를 누르세요
gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT3']))

### advanced 기능 추가하기

In [ ]:
def format_chat_prompt(message, chat_history, instruction):
    prompt = f"System:{instruction}"
    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

def respond(message, chat_history, instruction, temperature=0.7):
    prompt = format_chat_prompt(message, chat_history, instruction)
    chat_history = chat_history + [[message, ""]]
    stream = client.generate_stream(prompt,
                                      max_new_tokens=1024,
                                      stop_sequences=["\nUser:", "<|endoftext|>"],
                                      temperature=temperature)
                                      # 유저의 답변을 출력하지 않도록 stop_sequences 설정
    acc_text = ""
    # Streaming the tokens
    for idx, response in enumerate(stream):
            text_token = response.token.text

            if response.details:
                return

            if idx == 0 and text_token.startswith(" "):
                text_token = text_token[1:]

            acc_text += text_token
            last_turn = list(chat_history.pop(-1))
            last_turn[-1] += acc_text
            chat_history = chat_history + [last_turn]
            yield "", chat_history
            acc_text = ""

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) # 노트북 사이즈에 맞추기 위한 수치일 뿐입니다
    msg = gr.Textbox(label="Prompt")
    with gr.Accordion(label="Advanced options",open=False): # advanced 옵션 접어두기
        system = gr.Textbox(label="System message", lines=2, value="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.")
        temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1)
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot]) # 제출하려면 엔터를 누르세요
gr.close_all()
demo.queue().launch(share=True, server_port=int(os.environ['PORT4']))

In [ ]:
gr.close_all()